# Minesweeper !

## imports 

In [23]:
import pygame
import random
from queue import Queue
import time

## globals and intilization

In [24]:
WIDTH, HEIGHT = 625, 750
BG_COLOR = "white"
# ROWS, COLS = 25, 25
# BOMBS = 75
ROWS, COLS = 25, 25
BOMBS = 75
SIZE = WIDTH / ROWS

pygame.init()
NUM_FONT = pygame.font.SysFont('comicsans', 20)
LOST_FONT = pygame.font.SysFont('comicsans', 80)
TIME_FONT = pygame.font.SysFont('comicsans', 50)
NUM_COLORS = {1: "black", 2: "green", 3: "red", 4: "orange", 5: "yellow", 6: "purple", 7: "blue", 8: "pink"}

RECT_COLOR = (200, 200, 200)
CLICKED_RECT_COLOR = (140, 140, 140)
FLAG_RECT_COLOR = "green"
BOMB_COLOR = "red"

## functions

#### create mine field 

In [25]:
def create_mine_field(rows, cols, mines):
    field = [[0 for _ in range(cols)] for _ in range(rows)]
    mine_positions = set()

    while len(mine_positions) < mines:
        row = random.randrange(0, rows)
        col = random.randrange(0, cols)
        pos = row, col

        if pos in mine_positions:
            continue

        mine_positions.add(pos)
        field[row][col] = -1
    
    for mine in mine_positions:
        neighbors = get_neighbors(*mine, rows, cols)
        for r, c in neighbors:
            if field[r][c] != -1:
                field[r][c] += 1
    
    return field


#### draw

In [26]:
def draw(win, field, cover_field, current_time):
    win.fill(BG_COLOR)
    
    time_text = TIME_FONT.render(f"Time Elapsed: {round(current_time)}", 1, "black")
    win.blit(time_text, (10, HEIGHT - time_text.get_height()))

    for i, row in enumerate(field):
        y = SIZE * i
        for j, value in enumerate(row):
            x = SIZE * j
            
            is_covered = cover_field[i][j] == 0
            is_flag = cover_field[i][j] == -2
            is_bomb = value == -1
            if is_flag:
                pygame.draw.rect(win, FLAG_RECT_COLOR, (x, y, SIZE, SIZE))
                pygame.draw.rect(win, "black", (x, y, SIZE, SIZE), 1)
                continue

            if is_covered:
                pygame.draw.rect(win, RECT_COLOR, (x, y, SIZE, SIZE))
                pygame.draw.rect(win, "black", (x, y, SIZE, SIZE), 1)
                continue
            
            else:
                pygame.draw.rect(win, CLICKED_RECT_COLOR, (x, y, SIZE, SIZE))
                pygame.draw.rect(win, "black", (x, y, SIZE, SIZE), 1)
                if is_bomb:
                    pygame.draw.circle(win, BOMB_COLOR, (x + SIZE/2, y + SIZE/2), SIZE/2 - 4)


            if value > 0:
                text = NUM_FONT.render(str(value), 1, NUM_COLORS[value]) # the 1 means anti aliasing
                win.blit(text, (x + (SIZE/2 - text.get_width()/2), y + (SIZE/2 - text.get_height()/2)))


    pygame.display.update()

#### get neighbors

In [27]:
def get_neighbors(row, col, rows, cols):
    neighbors = []

    if row > 0: # UP
        neighbors.append((row - 1, col))
    if row < rows - 1: # DOWN
        neighbors.append((row + 1, col))
    if col > 0: # LEFT
        neighbors.append((row, col - 1))
    if col < cols - 1: # RIGHT
        neighbors.append((row, col + 1))

    if row > 0 and col > 0:
        neighbors.append((row - 1, col -1))
    if row < rows -1 and col < cols - 1:
        neighbors.append((row + 1, col+1))
    if row < rows - 1 and col > 0:
        neighbors.append((row + 1, col - 1))
    if row > 0 and col < cols - 1:
        neighbors.append((row - 1, col + 1))

    return neighbors

#### get grid pos from mouse click

In [28]:
def get_grid_pos(mouse_pos):
    mx, my = mouse_pos
    row = int(my // SIZE)
    col = int(mx // SIZE)

    return row, col

#### uncover from position

In [29]:
"""
uncovers all the neighboring tiles that are numbered and if one of the neighbors have a value of 0, finds the neighbors of those ones too and on and on...
"""
def uncover_from_pos(row, col, cover_field, field):
    q = Queue()
    q.put((row, col))
    visited = set()

    while not q.empty():
        current = q.get()

        neighbors = get_neighbors(*current, ROWS, COLS)
        for r, c in neighbors:
            if (r, c) in visited:
                continue

            value = field[r][c]
            if value == 0 and cover_field[r][c] != -2:
                q.put((r,c))
            if cover_field[r][c] != -2:
                cover_field[r][c] = 1

            visited.add((r, c))


#### draw end game text

In [30]:
def draw_game_end(win, text):
    text = LOST_FONT.render(text, 1, "black")
    win.blit(text, (WIDTH/2 - text.get_width()/2, HEIGHT/2 - text.get_height()/2))
    pygame.display.update()

#### check if game won

In [31]:
def check_for_win(cover_field):
    uncovered_count = 0

    for row in cover_field:
        for value in row:
            if value == 0 or value == -2:
                uncovered_count += 1

    # for row in range(ROWS):
    #     for col in range(COLS):
    #         if cover_field[row][col] != 0:
    #             uncovered_count += 1
    if uncovered_count == BOMBS:
        return True
    else:
        return False

## main function

In [32]:
win = pygame.display.set_mode((WIDTH, HEIGHT)) # intializing window
pygame.display.set_caption("Minesweeper")

run = True
field = create_mine_field(ROWS, COLS, BOMBS)
cover_field = [[0 for _ in range(COLS)] for _ in range(ROWS)]
flags = BOMBS
clicks = 0
lost = False
won = False

running = False

start_time = current_time = 0
while run:
    if start_time > 0:
        current_time = time.time() - start_time

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
            break

        if event.type == pygame.MOUSEBUTTONDOWN:
            row, col = get_grid_pos(pygame.mouse.get_pos())
            if row >= ROWS or col >= COLS:
                continue

            mouse_pressed = pygame.mouse.get_pressed()
            if mouse_pressed[0] and cover_field[row][col] != -2: # left mouse button
                cover_field[row][col] = 1

                if field[row][col] == -1: #bomb clicked, reset
                    lost = True

                if field[row][col] == 0:
                    uncover_from_pos(row, col, cover_field, field)
                
                if clicks == 0:
                    start_time = time.time()
                clicks += 1
                if check_for_win(cover_field):
                    won = True
                
            elif mouse_pressed[2] and cover_field[row][col] != 1:
                if cover_field[row][col] == -2:
                    cover_field[row][col] = 0
                    flags += 1
                else:
                    flags -= 1
                    cover_field[row][col] = -2
    if lost:
        draw(win, field, cover_field, current_time)
        draw_game_end(win, "you suck")
        pygame.time.delay(5000)

        field = create_mine_field(ROWS, COLS, BOMBS)
        cover_field = [[0 for _ in range(COLS)] for _ in range(ROWS)]
        flags = BOMBS
        clicks = 0
        lost = False
        start_time = 0

    elif won:
        draw(win, field, cover_field, current_time)
        draw_game_end(win, "wow you won!")
        pygame.time.delay(5000)

        field = create_mine_field(ROWS, COLS, BOMBS)
        cover_field = [[0 for _ in range(COLS)] for _ in range(ROWS)]
        flags = BOMBS
        clicks = 0
        won = False
        start_time = 0
    
    draw(win, field, cover_field, current_time)
pygame.quit()

### List Comprehension notes

**List Comprehensions**:
    [expr for val in collection]
    ^ first expression generates elements in the list
        follow it with a for loop over some collection of data
    [expr for val in collection if <test>]
    [expr for val in collection if <test1> and <test2>]

In [33]:
squares = [i**2 for i in range(1, 101)]
# print(squares)
remainders5 = [x**2 % 5 for x in range(1,101)]
# print(remainders5)
remainders11 = [x**2 %11 for x in range(1,101)]
# print(remainders11)

